In [3]:
from jupyter_plotly_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash
from dash.dependencies import Input, Output
from pymongo import MongoClient
import urllib.parse
from bson.json_util import dumps
import dash_leaflet as dl
import plotly.express as px
import dash_table as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from module5CRUD import AnimalShelter


app = JupyterDash('SimpleExample')
###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image

image_filename2 = 'sam.jpg' #I am little confused when replacing the line above the UI/UX document says 
#not to do that, so here is a second file showing my 'touch' on the project

encoded_image = base64.b64encode(open(image_filename, 'rb').read())

username = "aacuser"
password = "Buddy123!"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    html.Div(dcc.RadioItems(
                id='radio-items',
                options = [
                    {'label': 'Water Rescue', 'value': '1'},
                    {'label': 'Mountain Rescue', 'value': '2'},
                    {'label': 'Disaster Rescue', 'value': '3'},
                    {'label': 'Reset', 'value': '4'},

                    ],
                value = '4',
                labelStyle={'display': 'inline-block'}
    )), 
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        filter_action ="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        

    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',

                    ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )
            ]
        )
]
)

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




@app.callback(Output('datatable-id','data'),
    [Input('radio-items', 'value')])

def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if (int(filter_type) == 4):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))
    # use higher number of button clicks to determine filter type, can you think of a better way ? ....
    elif (int(filter_type) == 1):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif (int(filter_type) == 2):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif (int(filter_type) == 3):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))
    
    #columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    #data=df.to_dict('records')
        
        
    return df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, values=values, names=names)
        )    
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
    

app

ImportError: No module named jupyter_plotly_dash